# Fashion-MNIST-Clean
### Image Translation

#### Imports

In [1]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model
from DatasetsNew import FashionMNIST

import cox.store
import torch as ch
import DatasetsNew
import defaultsNew
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np

import pickle
import sys
import psutil
import gc


from label_maps import CLASS_DICT
from user_constants import DATA_PATH_DICT

In [2]:
NUM_WORKERS = 8
BATCH_SIZE = 100

In [3]:
DATA = 'FashionMNIST'

#### Load dataset

In [4]:
# Load dataset
dataset_function = getattr(DatasetsNew, 'FashionMNIST')
DATA_PATH_DICT[DATA]
dataset = dataset_function(DATA_PATH_DICT[DATA])
_, test_loader = dataset.make_loaders(workers=NUM_WORKERS,
                                      batch_size=BATCH_SIZE, 
                                      data_aug=False)
data_iterator = enumerate(test_loader.dataset)

==> Preparing dataset fashionmnist..


In [5]:
model, _ = make_and_restore_model(arch='resnet18', dataset=dataset,resume_path='models/Fashion_mnist_Clean_100_epochs_checkpoint.pt.best',parallel=False)
model.eval()
print()

=> loading checkpoint 'models/Fashion_mnist_Clean_100_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/Fashion_mnist_Clean_100_epochs_checkpoint.pt.best' (epoch 51)



In [6]:
_, (imgs, label) = next(enumerate(test_loader))

In [7]:
# EXPERIMENTAL
kwargs = {
        #'criterion': ch.nn.CrossEntropyLoss(reduction='none'),
        'constraint':'2',
        'eps': 100,
        'step_size': 1.5,
        'iterations': 75,
        'do_tqdm': True,
        'targeted': True,
}

### Translation for clean unpoisoned model

In [8]:
model, _ = make_and_restore_model(arch='resnet18', dataset=dataset,resume_path='models/Fashion_mnist_Clean_100_epochs_checkpoint.pt.best',parallel=False)
model.eval()
print()

=> loading checkpoint 'models/Fashion_mnist_Clean_100_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/Fashion_mnist_Clean_100_epochs_checkpoint.pt.best' (epoch 51)



In [9]:
_, train_loader = dataset.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

for targ_lbl in range(10):
    targ = []


    for i in range(BATCH_SIZE):
        targ.append(targ_lbl)
    targ = ch.tensor(targ)

    _, img_translated = model(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)

    for i in range(4):
        _, (imgs, label) = next(enumerate(train_loader))
        targ = []
        for i in range(BATCH_SIZE):
            targ.append(targ_lbl)
        targ = ch.tensor(targ)

        _, img_translated_new = model(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)
        img_translated = ch.cat((img_translated, img_translated_new), 0)

    print(img_translated.size())
    
    
    filename = "saved_pickles/clean-model-img-translated/img_translated_from_dataset_" + str(targ_lbl) + ".pkl"
    
    with open(filename, 'wb') as handle:
        pickle.dump(img_translated, handle)
    print("saved: ", filename)

==> Preparing dataset fashionmnist..


Current loss: 9.353506088256836:   1%|▏         | 1/75 [00:00<00:07,  9.58it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_0.pkl


Current loss: 1.9502657651901245:   3%|▎         | 2/75 [00:00<00:03, 18.64it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_1.pkl


...

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_7.pkl


Current loss: 9.729087829589844:   1%|▏         | 1/75 [00:00<00:08,  8.49it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_8.pkl


Current loss: 0.0: 100%|██████████| 75/75 [00:07<00:00,  9.91it/s]                   


torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_9.pkl
